# Clean-up Resource

# 1. 세이지 메이커 실험 지우기

In [2]:
import time

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent

In [3]:
experiment_name='NCFModel-single-train'  #<== 생성한 experiment_name을 넣어야 합니다.

In [4]:
def cleanup_sme_sdk(experiment):
    for trial_summary in experiment.list_trials():
        trial = Trial.load(trial_name=trial_summary.trial_name)
        for trial_component_summary in trial.list_trial_components():
            tc = TrialComponent.load(
                trial_component_name=trial_component_summary.trial_component_name)
            trial.remove_trial_component(tc)
            try:
                # comment out to keep trial components
                tc.delete()
            except:
                # tc is associated with another trial
                continue
            # to prevent throttling
            time.sleep(.5)
        trial.delete()
        experiment_name = experiment.experiment_name
    experiment.delete()
    print(f"\nExperiment {experiment_name} deleted")

In [ ]:
experiment_to_cleanup = Experiment.load(
    # Use experiment name not display name
    experiment_name=experiment_name)

cleanup_sme_sdk(experiment_to_cleanup)

# 2. S3 버킷 삭제

In [8]:
import sagemaker
sagemaker_session = sagemaker.session.Session()

In [9]:
bucket = sagemaker_session.default_bucket()
bucket

'sagemaker-us-east-1-917939256726'

In [12]:
!aws s3 ls s3://$bucket/NCFModel --recursive

2022-05-27 11:29:36    2891424 NCFModel/data/ml-1m.test.negative
2022-05-27 11:29:36     128039 NCFModel/data/ml-1m.test.rating
2022-05-27 11:29:36   20982911 NCFModel/data/ml-1m.train.rating


In [10]:
# !aws s3 rm s3://$bucket/NCFModel --recursive

In [13]:
!aws s3 ls s3://$bucket/NCFModel --recursive

2022-05-27 11:29:36    2891424 NCFModel/data/ml-1m.test.negative
2022-05-27 11:29:36     128039 NCFModel/data/ml-1m.test.rating
2022-05-27 11:29:36   20982911 NCFModel/data/ml-1m.train.rating


# 3. 세이지 메이커 앤드포인트 삭제 확인
- 아래와 같이 콘솔에 가셔서 엔드포인트가 없어야 합니다.
    - 만일 존재하면 삭제 해주세요.

![sm_endpoint_console.png](img/sm_endpoint_console.png)

# 4. 세이지 메이커 노트북 인스턴스 삭제

## 아래와 같이 노트북 인스턴스를 선택하고, Stop (중지) 후에 Delete(삭제) 해주세요.

![notebook_instance.png](img/notebook_instance.png)